In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Config import gkey

### Store Part I results into DataFrame

- Load the csv exported in Part I to a DataFrame

In [3]:
saved_df = pd.read_csv('random_city_weather.csv')
saved_df

,city,country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed
0,punta arenas,CL,-53.1500,-70.9167,46.20,81,90,19.57
1,busselton,AU,-33.6500,115.3333,78.51,48,0,4.00
2,ilulissat,GL,69.2167,-51.1000,-0.40,55,0,4.61
3,atuona,PF,-9.8000,-139.0333,78.46,80,11,17.34
4,sao jose da coroa grande,BR,-8.8978,-35.1478,77.88,83,100,11.10
...,...,...,...,...,...,...,...,...
581,west wendover,US,40.7391,-114.0733,39.20,65,90,8.05
582,bonthe,SL,7.5264,-12.5050,77.20,88,3,3.74
583,manokwari,ID,-0.8667,134.0833,83.10,64,89,2.86
584,casper,US,42.8666,-106.3131,37.20,52,1,12.66


### Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [12]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

#get the coordinates
coordinates = saved_df[['Lat', 'Lng']].reset_index(drop=True)
coordinates
humidity_data = saved_df['Humidity']

In [20]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# 
temperature_df = saved_df[saved_df['Temperature'] >= 70]
ideal_temp_df = temperature_df[temperature_df['Temperature'] < 80]
ideal_ws =  ideal_temp_df[ideal_temp_df['Wind Speed'] < 5]
ideal_cloudiness = ideal_ws.loc[(ideal_ws['Cloudiness'] == 0) & (ideal_ws['Humidity'] < 75)]
perfect_places = ideal_cloudiness.reset_index()

In [48]:
#get coordinates of the ideal locations
hotel_df = perfect_places[['city', 'country', 'Lat', 'Lng',]]
hotel_df['Hotel Name'] = ''
hotel_df

<ipython-input-48-ac0cdca0e967>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,city,country,Lat,Lng,Hotel Name
0,busselton,AU,-33.6500,115.3333,
1,bassar,TG,9.2500,0.7833,
2,patiya,BD,22.2931,91.9762,
3,tupaciguara,BR,-18.5922,-48.7050,
4,piopio,NZ,-38.4667,175.0167,
5,keetmanshoop,NaN,-26.5833,18.1333,


In [50]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)




In [53]:
hotel_df['Hotel Name'] = hotel_name
hotel_df

<ipython-input-53-4778c64b5449>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_name


,city,country,Lat,Lng,Hotel Name
0,busselton,AU,-33.6500,115.3333,The Sebel Busselton
1,bassar,TG,9.2500,0.7833,HOTEL BASSAR
2,patiya,BD,22.2931,91.9762,NaN
3,tupaciguara,BR,-18.5922,-48.7050,Hotel Nossa Senhora De Aparecida
4,piopio,NZ,-38.4667,175.0167,Owls Nest Motel
5,keetmanshoop,NaN,-26.5833,18.1333,Hotel Schützenhaus


In [56]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))